In [1]:
%load_ext autoreload
%autoreload 2

# System imports
import sys
import os

# Add the parent directory of 'notebooks' to sys.path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), "../.."))  # Move one level up
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [ ]:
import torch
from pathlib import Path
from solver import Solver
from griddy.griddy_tuna import hit_griddy, SearchMethod
from models.ChromaTransformer import ChromaTransformerModel
from data.data_loader import MirDataProcessor
from utils.model_utils import get_device
from solver import TrialMetric

In [3]:
device = get_device()
print(f"Device is {device}")

Device is cuda


In [ ]:
# Initialize the data processor and create data loaders
data_processor = MirDataProcessor(batch_size=256, process_sequential=True, seq_length=16, overlap_sequence=True, use_median=True)
train_loader, test_loader, num_classes = data_processor.build_data_loaders(device="cuda")

Preparing model data...
Loading the combined CSV file...
Separating song IDs, features, and labels...
Scaling features using MinMaxScaler...
Encoding labels using LabelEncoder...
Saving the scaler to c:\Users\maxwe\OneDrive\Documents\amadeus-ex-machina\data\processed\scaler.pkl...
Saving the label encoder to c:\Users\maxwe\OneDrive\Documents\amadeus-ex-machina\data\processed\label_encoder.pkl...
Creating sequences of chromagram data within song boundaries...
Splitting data into training and testing sets...
Data preparation complete.
Number of classes determined: 976
Creating TensorDatasets for training and testing data...
Creating DataLoaders for training and testing datasets...
Data loaders are ready for training and testing.


In [ ]:
# NOTE: Do not assume these values are anything but trash, they're just here for testing

SOLVER_PARAMS = {
    Solver : {
        "device": device,
        "batch_size": 256,
        "epochs": [5, 10],
        "early_stop_epochs": 3, # early stop after n epochs without improvement, 0 to disable
        "warmup_epochs": 0, # 0 to disable
        "dtype": "float16",
        "train_dataloader": train_loader, # must be DataLoader object
        "valid_dataloader": test_loader, # must be DataLoader object
        "direction": "minimize" # must specify this, even if not used by solver
    }
}

MODEL_PARAMS = {
    ChromaTransformerModel: {
        "input_dim": [24],
        "seq_length": [16],
        "num_classes": [num_classes],
        "dim_feedworward": [256, 512, 1024],
        "d_model": [128, 256],
        "num_layers": [4, 6]
    }
}

OPTIM_PARAMS = {
    torch.optim.SGD : {
        "lr": [0.001, 0.1, SearchMethod.LOG_UNIFORM],
        "momentum": [0.9, 0.99, SearchMethod.UNIFORM],
        "weight_decay": [0.00001],
    },
    torch.optim.Adam : {
        "lr": [0.008, 0.005, 0.001], # this will auto-search as CATEGORICAL
    }
}

SCHED_PARAMS = {
    torch.optim.lr_scheduler.CosineAnnealingWarmRestarts : {
        "T_0": [10],
    },
    torch.optim.lr_scheduler.StepLR : {
        "step_size": [5, 10, 15],
        "gamma" : [0.1, 0.05],
    }
}

CRITERION_PARAMS = {
    torch.nn.CrossEntropyLoss : {}
}

PARAM_SET = {
    "solver": SOLVER_PARAMS,
    "model" : MODEL_PARAMS,
    "optim" : OPTIM_PARAMS,
    "sched" : SCHED_PARAMS,
    "criterion" : CRITERION_PARAMS,
}

In [ ]:
my_study = "chroma_transformer"

output_folder = Path("griddy")

solver_reference = hit_griddy(my_study, param_set=PARAM_SET, out_dir=output_folder, n_trials=30, n_jobs=1, prune=False, resume=False, trial_metric=TrialMetric.ACCURACY)
# NOTE: modest values of n_trials and n_jobs set here for testing, set your values accordingly

"Hitting the griddy..." -Ellie


[I 2024-12-08 17:46:29,229] A new study created in RDB with name: chroma_transformer
c:\Users\maxwe\OneDrive\Documents\amadeus-ex-machina\griddy\griddy_tuna.py:127: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  return trial.suggest_loguniform(name, min(values), max(values))
c:\Users\maxwe\OneDrive\Documents\amadeus-ex-machina\griddy\griddy_tuna.py:125: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  return trial.suggest_uniform(name, min(values), max(values))


-----------------------------------
Epoch 1
-----------------------------------


Training:   0%|          | 0/12893 [00:00<?, ?it/s]

Training Loss: 1.8452. Validation Loss: 1.4233.
Training Accuracy: 0.5426. Validation Accuracy: 0.6201.
-----------------------------------
Epoch 2
-----------------------------------


c:\Users\maxwe\anaconda3\envs\amadeus-ex-machina\Lib\site-packages\torch\optim\lr_scheduler.py:152: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Training:   0%|          | 0/12893 [00:00<?, ?it/s]